In [19]:
using ShipMMG
using JuMP
using Ipopt
using CSV, DataFrames, ProgressMeter, Distributions

In [20]:
# 船体に働く流体力
function X_H(u, v, r)
    U = sqrt(u^2 + (v - r * x_G)^2)
    β = asin(-(v - r * x_G) / U)
    r_dash = r * L_pp / U
    v_dash = v / U
    return 0.5 * ρ * L_pp * d * U^2 * X_H_dash(v_dash, r_dash)
end

function Y_H(u, v, r)
    U = sqrt(u^2 + (v - r * x_G)^2)
    β = asin(-(v - r * x_G) / U)
    r_dash = r * L_pp / U
    v_dash = v / U
    return 0.5 * ρ * L_pp * d * U^2 * Y_H_dash(v_dash, r_dash)
end

function N_H(u, v, r)
    U = sqrt(u^2 + (v - r * x_G)^2)
    β = asin(-(v - r * x_G) / U)
    r_dash = r * L_pp / U
    v_dash = v / U
    return 0.5 * ρ * L_pp^2 * d * U^2 * N_H_dash(v_dash, r_dash)
end

# 船体に働く流体力(無次元)
function X_H_dash(β, r_dash)
    return -R_0_dash + X_vv_dash*β^2 + X_vr_dash*β*r_dash + X_rr_dash*r_dash^2 + X_vvvv_dash*β^4
end

function Y_H_dash(β, r_dash)
    return Y_v_dash*β + Y_r_dash*r_dash + Y_vvr_dash*β^2*r_dash + Y_vrr_dash*β*r_dash^2 + Y_vvv_dash*β^3 + Y_rrr_dash*r_dash^3
end

function N_H_dash(β, r_dash)
    return N_v_dash*β + N_r_dash*r_dash + N_vvr_dash*β^2*r_dash + N_vrr_dash*β*r_dash^2 + N_vvv_dash*β^3 + N_rrr_dash*r_dash^3
end

# 舵に働く力
function X_R(u, v, r, δ, n_p)
    return -(1 - t_R) * F_N(u, v, r, δ, n_p) * sin(δ)
end

function Y_R(u, v, r, δ, n_p)
    return -(1 + a_H) * F_N(u, v, r, δ, n_p) * cos(δ)
end

function N_R(u, v, r, δ, n_p)
    return -(x_R + a_H * x_H) * F_N(u, v, r, δ, n_p) * cos(δ)
end

function F_N(u, v, r, δ, n_p)
    U = sqrt(u^2 + (v - r * x_G)^2)
    β = asin(-(v - r * x_G) / U)
    r_dash = r * L_pp / U
    w_P = w_P0 * exp(-4.0 * (β - x_P * r_dash)^2)
    u_p = (1 - w_P) * u
    u_R = ϵ * u_p * sqrt(η * (1 + κ * (sqrt(1 + (8 * K_T(u, v, r, n_p)) / (π * J(u, v, r, n_p)^2)) - 1))^2 + (1 - η))
    β_R = β - l_R * r_dash
    if β_R < 0.0
        γ_R = γ_R_minus
    else
        γ_R = γ_R_plus
    end 
    v_R = U * γ_R * (β - l_r_dash * r_dash)
    α_R = δ - atan(v_R, u_R)
    U_R = sqrt(u_R^2 + v_R^2)
    return 0.5 * ρ * A_R * U_R^2 * f_α * sin(α_R)
end

# プロペラに働く力
function X_P(u, v, r, δ, n_p)
    return (1 - t_P) * T_P(u, v, r, n_p)
end

function K_T(u, v, r, n_p)
    return k_0 + k_1 * J(u, v, r, n_p) + k_2 * J(u, v, r, n_p)^2
end

function J(u, v, r, n_p)
    U = sqrt(u^2 + (v - r * x_G)^2)
    β = atan(-v, u)
    r_dash = r * L_pp / U
    w_P = w_P0 * exp(-4.0 * (β - x_P * r_dash)^2)
    return u * (1 - w_P) / (n_p * D_p)
end

function T_P(u, v, r, n_p)
    return K_T(u, v, r, n_p) * ρ * n_p^2 * D_p^4
end

T_P (generic function with 1 method)

In [21]:
# パラメータを定義
const ρ = 1025.0
const L_pp = 3.50
const B = 0.57  
const d = 0.16  
const nabla = 0.7407 * L_pp * B * d 
const x_G = 0.0112 
const D_p = 0.03373 * L_pp 
const m = nabla * ρ 
const I_zG = m * ((0.25 * L_pp)^2)
const A_R = 0.0189  
const η = 0.8023  
const m_x = 0.0475 * m 
const m_y = 0.6739 * m 
const J_z = 0.0306 * m * (L_pp^2) 
const f_α = 2.634 
const ϵ = 1.345
const t_R = 0.084
const a_H = 0.054
const x_H = -0.388 * L_pp
const γ_R_minus = 0.225 
const γ_R_plus = 0.574 
const l_r_dash = -0.641 
const l_R = -0.641  
const κ = 0.482
const t_P = 0.180 
const w_P0 = 0.40  
const x_P = -0.490 
const x_R = -0.500 * L_pp

const k_0 = 0.3516
const k_1 = -0.3624
const k_2 = -0.1100

-0.11

In [22]:
R_0_dash = 0.0160
X_vv_dash = -0.0639
X_vr_dash = 0.0813
X_rr_dash = -0.0141
X_vvvv_dash = 0.4563
Y_v_dash = -0.2969
Y_r_dash = 0.0576
Y_vvv_dash = -0.8428
Y_vvr_dash = 0.1656
Y_vrr_dash = -0.2045
Y_rrr_dash = 0.0176
N_v_dash = -0.0878
N_r_dash = -0.0422
N_vvv_dash = -0.0426
N_vvr_dash = -0.2290
N_vrr_dash = -0.0165
N_rrr_dash = -0.0157

-0.0157

In [23]:
function mpc(f_x_t::Vector{Float64}, u1_t::Float64, u2_t::Float64, f_obs::DataFrame ,t::Int, N::Int)
    # モデル作成
    model = Model(Ipopt.Optimizer) # Ipoptを使用
    set_silent(model) # 計算の進行状況を非表示
    L_f = 1.0 # センサー位置(船首)
    L_t = 1.0 # センサー位置(船尾)
    Q = 1.0 # 座標誤差への重み
    R1 = 1.0 # 制御量変化(δ)への重み
    R2 = 1.0 # 制御量変化(np)への重み

    # 方程式を登録
    register(model, :X_H, 3, X_H; autodiff = true)
    register(model, :Y_H, 3, Y_H; autodiff = true)
    register(model, :N_H, 3, N_H; autodiff = true)
    register(model, :X_H_dash, 2, X_H_dash; autodiff = true)
    register(model, :Y_H_dash, 2, Y_H_dash; autodiff = true)
    register(model, :N_H_dash, 2, N_H_dash; autodiff = true)
    register(model, :X_R, 5, X_R; autodiff = true)
    register(model, :Y_R, 5, Y_R; autodiff = true)
    register(model, :N_R, 5, N_R; autodiff = true)
    register(model, :F_N, 5, F_N; autodiff = true)
    register(model, :X_P, 5, X_P; autodiff = true)
    register(model, :K_T, 4, K_T; autodiff = true)
    register(model, :J, 4, J; autodiff = true)
    register(model, :T_P, 4, T_P; autodiff = true)
    
    # 使用する変数を設定
    x = @variable(model, [t:t+N+1, 1:10]) # 操作量 x
    u1 = @variable(model, [t:t+N, 1:1]) # 制御量 u1
    u2 = @variable(model, [t:t+N, 1:1]) # 制御量 u2

    # 制御量の制約条件
    @variable(model, 0.0 <= u1[t:t+N, 1:1] <= 1.0) 
    @variable(model, 0.0 <= u2[t:t+N, 1:1] <= 40.0)
    
    # 操作量の箱を作成
    for k in t:t+N+1
        set_start_value.(x[k, 1], f_x_t[1])
        set_start_value.(x[k, 2], f_x_t[2])
        set_start_value.(x[k, 3], f_x_t[3])
        set_start_value.(x[k, 4], f_x_t[4])
        set_start_value.(x[k, 5], f_x_t[5])
        set_start_value.(x[k, 6], f_x_t[6])
        set_start_value.(x[k, 7], f_x_t[7])
        set_start_value.(x[k, 8], f_x_t[8])
        set_start_value.(x[k, 9], f_x_t[9])
        set_start_value.(x[k, 10], f_x_t[10])  
    end
    
    # 初期状態(t)での変数xの設定
    for i in 1:10
        @constraint(model, x[t, i] == f_x_t[i])
    end

    # MMGを用いた制約
    for k in t:t+N
        @NLconstraint(model, x[k+1,1] == x[k,1] + dt * ((X_H(x[k,1], x[k,2], x[k,3]) + X_R(x[k,1], x[k,2], x[k,3], u1[k,1], u2[k,1]) + X_P(x[k,1], x[k,2], x[k,3], u1[k,1], u2[k,1]) + (m+m_y)*x[k,2]*x[k,3] + x_G*m*(x[k,3])^2) / (m+m_x)))
        @NLconstraint(model, x[k+1,2] == x[k,2] + dt * ((x_G^2) * (m^2) * x[k,1] * x[k,3] - (N_H(x[k,1], x[k,2], x[k,3])+N_R(x[k,1], x[k,2], x[k,3], u1[k,1], u2[k,1])) * x_G * m +((Y_H(x[k,1], x[k,2], x[k,3]) + Y_R(x[k,1], x[k,2], x[k,3], u1[k,1], u2[k,1])) - (m + m_x) * x[k,1] * x[k,3]) * (I_zG + J_z + (x_G^2) * m)) / ((I_zG + J_z + (x_G^2) * m) * (m + m_y) - (x_G^2) * (m^2)))
        @NLconstraint(model, x[k+1,3] == x[k,3] + dt * (N_H(x[k,1], x[k,2], x[k,3]) + N_R(x[k,1], x[k,2], x[k,3], u1[k,1], u2[k,1]) - x_G * m * (((x_G^2) * (m^2) * x[k,1] * x[k,3] - (N_H(x[k,1], x[k,2], x[k,3])+N_R(x[k,1], x[k,2], x[k,3], u1[k,1], u2[k,1])) * x_G * m +((Y_H(x[k,1], x[k,2], x[k,3]) + Y_R(x[k,1], x[k,2], x[k,3], u1[k,1], u2[k,1])) - (m + m_x) * x[k,1] * x[k,3]) * (I_zG + J_z + (x_G^2) * m)) / ((I_zG + J_z + (x_G^2) * m) * (m + m_y) - (x_G^2) * (m^2)) + x[k,1] * x[k,3])) / (I_zG + J_z + (x_G^2) * m))
        @NLconstraint(model, x[k+1,4] == x[k,4] + dt * (x[k,1] * cos(x[k,6]) - x[k,2] * sin(x[k,6])))
        @NLconstraint(model, x[k+1,5] == x[k,5] + dt * (x[k,1] * sin(x[k,6]) + x[k,2] * cos(x[k,6])))
        @NLconstraint(model, x[k+1,6] == x[k,6] + dt * (x[k,3]))
        @NLconstraint(model, x[k+1,7] == x[k+1,4] + L_f * cos(x[k+1,6]))
        @NLconstraint(model, x[k+1,8] == x[k+1,5] + L_f * sin(x[k+1,6]))
        @NLconstraint(model, x[k+1,9] == x[k+1,4] + L_t * cos(x[k+1,6] + pi))
        @NLconstraint(model, x[k+1,10] == x[k+1,5] + L_t * sin(x[k+1,6] + pi))
    end
    # 制御量変化の制約
    if t != 1
        @NLconstraint(model, -0.02 <= (u1[t,1] - u1_t) <= 0.02)
        @NLconstraint(model, -1.0 <= (u2[t,1] - u2_t) <= 1.0)
    end
    for k in t:t+N-1
        @NLconstraint(model, -0.02 <= (u1[k+1,1] - u1[k,1]) <= 0.02)
        @NLconstraint(model, -1.0 <= (u2[k+1,1] - u2[k,1]) <= 1.0)
    end

    #評価関数の設定
    @NLobjective(model, Min, sum((Q*((x[k,7]-f_obs[k,12])^2 +(x[k,8]-f_obs[k,13])^2 +(x[k,9]-f_obs[k,14])^2 +(x[k,10]-f_obs[k,15])^2 + (x[t+N+1,7]-f_obs[t+N+1,12])^2 +(x[t+N+1,8]-f_obs[t+N+1,13])^2 +(x[t+N+1,9]-f_obs[t+N+1,14])^2 +(x[t+N+1,10]-f_obs[t+N+1,15])^2)+R1*((u1[k,1]-u1[k-1,1])^2) + R2*((u2[k,1]-u2[k-1,1])^2)) for k in t+1:t+N))
    
    # 最適化の実行
    optimize!(model)
    
    # 次点の操作量x(t+1)とそれに用いた制御量u(t)の値を返す
    return value.(x[t+1,1:10]), value.(u1[t,1]), value.(u2[t,1]) # 最適化後の入力x・制御量u
end

mpc (generic function with 1 method)

In [24]:
function realMMG(x_0::Vector{Float64}, u1_val::Float64, u2_val::Float64)
    global R_0_dash, X_vv_dash, X_vr_dash, X_rr_dash, X_vvvv_dash
    global Y_v_dash, Y_r_dash, Y_vvv_dash, Y_vvr_dash, Y_vrr_dash, Y_rrr_dash
    global N_v_dash, N_r_dash, N_vvv_dash, N_vvr_dash, N_vrr_dash, N_rrr_dash
    
    L_f = 1.0 # センサー位置(船首)
    L_t = 1.0 # センサー位置(船尾)

    R_0_dash = 0.020
    X_vv_dash = -0.0799
    X_vr_dash = 0.1016
    X_rr_dash = -0.0176
    X_vvvv_dash = 0.5704
    Y_v_dash = -0.3711
    Y_r_dash = 0.0720
    Y_vvv_dash = -1.0535
    Y_vvr_dash = 0.207
    Y_vrr_dash = -0.2556
    Y_rrr_dash = 0.0220
    N_v_dash = -0.1097
    N_r_dash = -0.0527
    N_vvv_dash = -0.0533
    N_vvr_dash = -0.2863
    N_vrr_dash = -0.0206
    N_rrr_dash = -0.0196
    
    u_real = rand(Normal(0.0, 0.01)) + (x_0[1] + dt * ((X_H(x_0[1], x_0[2], x_0[3]) + X_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1]) + X_P(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1]) + (m+m_y)*x_0[2]*x_0[3] + x_G*m*(x_0[3])^2) / (m+m_x)))
    v_real = rand(Normal(0.0, 0.01)) + (x_0[2] + dt * (((x_G^2) * (m^2) * x_0[1] * x_0[3] - (N_H(x_0[1], x_0[2], x_0[3])+N_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1])) * x_G * m +((Y_H(x_0[1], x_0[2], x_0[3]) + Y_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1])) - (m + m_x) * x_0[1] * x_0[3]) * (I_zG + J_z + (x_G^2) * m)) / ((I_zG + J_z + (x_G^2) * m) * (m + m_y) - (x_G^2) * (m^2))))
    r_real = rand(Normal(0.0, 0.001)) + (x_0[3] + dt * ((N_H(x_0[1], x_0[2], x_0[3]) + N_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1]) - x_G * m * (((x_G^2) * (m^2) * x_0[1] * x_0[3] - (N_H(x_0[1], x_0[2], x_0[3])+N_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1])) * x_G * m +((Y_H(x_0[1], x_0[2], x_0[3]) + Y_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1])) - (m + m_x) * x_0[1] * x_0[3]) * (I_zG + J_z + (x_G^2) * m)) / ((I_zG + J_z + (x_G^2) * m) * (m + m_y) - (x_G^2) * (m^2)) + x_0[1] * x_0[3])) / (I_zG + J_z + (x_G^2) * m)))
    x_real = x_0[4] + dt * (x_0[1] * cos(x_0[6]) - x_0[2] * sin(x_0[6]))
    y_real = x_0[5] + dt * (x_0[1] * sin(x_0[6]) + x_0[2] * cos(x_0[6]))
    ψ_real = x_0[6] + dt * (x_0[3])
    x1_real = x_real + L_f * cos(ψ_real)
    y1_real = y_real + L_f * sin(ψ_real)
    x2_real = x_real + L_t * cos(ψ_real + pi)
    y2_real = y_real + L_t * sin(ψ_real + pi)
    
    return u_real, v_real, r_real, x_real, y_real, ψ_real, x1_real, y1_real, x2_real, y2_real
end     

realMMG (generic function with 1 method)

In [25]:
function MMG_error(p::Vector{Float64}, x_0::Vector{Float64}, u1_val::Float64, u2_val::Float64, u_real::Float64, v_real::Float64, r_real::Float64)
    global R_0_dash, X_vv_dash, X_vr_dash, X_rr_dash, X_vvvv_dash
    global Y_v_dash, Y_r_dash, Y_vvv_dash, Y_vvr_dash, Y_vrr_dash, Y_rrr_dash
    global N_v_dash, N_r_dash, N_vvv_dash, N_vvr_dash, N_vrr_dash, N_rrr_dash
    
    R_0_dash = p[1]
    X_vv_dash = p[2]
    X_vr_dash = p[3]
    X_rr_dash = p[4]
    X_vvvv_dash = p[5]
    Y_v_dash = p[6]
    Y_r_dash = p[7]
    Y_vvv_dash = p[8]
    Y_vvr_dash = p[9]
    Y_vrr_dash = p[10]
    Y_rrr_dash = p[11]
    N_v_dash = p[12]
    N_r_dash = p[13]
    N_vvv_dash = p[14]
    N_vvr_dash = p[15]
    N_vrr_dash = p[16]
    N_rrr_dash = p[17]
    
    u = x_0[1] + dt * ((X_H(x_0[1], x_0[2], x_0[3]) + X_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1]) + X_P(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1]) + (m+m_y)*x_0[2]*x_0[3] + x_G*m*(x_0[3])^2) / (m+m_x))
    v = x_0[2] + dt * (((x_G^2) * (m^2) * x_0[1] * x_0[3] - (N_H(x_0[1], x_0[2], x_0[3])+N_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1])) * x_G * m +((Y_H(x_0[1], x_0[2], x_0[3]) + Y_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1])) - (m + m_x) * x_0[1] * x_0[3]) * (I_zG + J_z + (x_G^2) * m)) / ((I_zG + J_z + (x_G^2) * m) * (m + m_y) - (x_G^2) * (m^2)))
    r = x_0[3] + dt * ((N_H(x_0[1], x_0[2], x_0[3]) + N_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1]) - x_G * m * (((x_G^2) * (m^2) * x_0[1] * x_0[3] - (N_H(x_0[1], x_0[2], x_0[3])+N_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1])) * x_G * m +((Y_H(x_0[1], x_0[2], x_0[3]) + Y_R(x_0[1], x_0[2], x_0[3], u1_val[1], u2_val[1])) - (m + m_x) * x_0[1] * x_0[3]) * (I_zG + J_z + (x_G^2) * m)) / ((I_zG + J_z + (x_G^2) * m) * (m + m_y) - (x_G^2) * (m^2)) + x_0[1] * x_0[3])) / (I_zG + J_z + (x_G^2) * m))
    
    return (u_real-u)^2+(v_real-v)^2+(r_real-r)^2
end   

MMG_error (generic function with 1 method)

In [26]:
function run_mpc(duration::Int, dt::Float64)
    global R_0_dash, X_vv_dash, X_vr_dash, X_rr_dash, X_vvvv_dash
    global Y_v_dash, Y_r_dash, Y_vvv_dash, Y_vvr_dash, Y_vrr_dash, Y_rrr_dash
    global N_v_dash, N_r_dash, N_vvv_dash, N_vvr_dash, N_vrr_dash, N_rrr_dash

    #観測データ
    obs = CSV.read("./observation/suzaku_sim_data.csv", DataFrame)
        
    # 初期値設定
    x_0 = Vector(Float64[0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0, 0.0])
    u1_0 = 0.0
    u2_0 = 0.0

    # データフレーム
    cols = Int(duration/dt) + 1
    df_x = DataFrame(
        time = collect(0:dt:duration),
        u = zeros(cols),
        v = zeros(cols),
        r = zeros(cols),
        x = zeros(cols),
        y = zeros(cols),
        ψ = zeros(cols),
        x1 = zeros(cols),
        y1 = zeros(cols),
        x2 = zeros(cols),
        y2 = zeros(cols),
        δ = zeros(cols),
        n_p = zeros(cols),
        R_0_dash = zeros(cols)
    )

    #初期値をデータフレームへ
    df_x[1, 2:11] .= x_0
    df_x[1, 12:13] .= [u1_0, u2_0]

    # MPCの実行
    @showprogress for t in 1:Int(duration/dt)
        #MPCの結果
        x_val, u1_val, u2_val = mpc(x_0, u1_0, u2_0, obs, t, N)

        #制御量を真モデルに反映(実際の挙動)
        u_real, v_real, r_real, x_real, y_real, ψ_real, x1_real, y1_real, x2_real, y2_real = realMMG(x_0, u1_val[1], u2_val[1])
        
        #MPC用モデルの微係数セット
        params = [
            [0.0160, -0.0639, 0.0813, -0.0141, 0.4563, -0.2969, 0.0576, -0.8428, 0.1656, -0.2045, 0.0176, -0.0878, -0.0422, -0.0426, -0.2290, -0.0165, -0.0157],
            [0.0190, -0.0759, 0.0965, -0.0167, 0.5419, -0.3525, 0.0684, -1.0008, 0.1966, -0.2428, 0.0209, -0.1042, -0.0501, -0.0506, -0.2720, -0.0196, -0.0186],
            [0.0220, -0.0879, 0.1118, -0.0194, 0.6274, -0.4082, 0.0792, -1.1589, 0.2277, -0.2812, 0.0242, -0.1207, -0.0580, -0.0586, -0.3149, -0.0227, -0.0216],
            [0.0230, -0.0919, 0.1168, -0.0202, 0.6560, -0.4268, 0.0828, -1.2115, 0.2380, -0.2939, 0.0253, -0.1262, -0.0606, -0.0613, -0.3292, -0.0237, -0.0225]
        ]

        #微係数セットを用いたMMGと実際の挙動の誤差
        results = [(MMG_error(p, x_0, u1_val[1], u2_val[1], u_real, v_real, r_real), p) for p in params]
        
        #誤差が最小となる微係数セットの探索
        minimum_error = findmin(results)

        #結果をデータフレームへ
        df_x[t+1, 2:11] .= [u_real, v_real, r_real, x_real, y_real, ψ_real, x1_real, y1_real, x2_real, y2_real]
        df_x[t+1, 12:14] .= [u1_val, u2_val, minimum_error[1][2][1]]
        
        #次のMPCに用いる初期値へ設定
        x_0 = Vector([u_real, v_real, r_real, x_real, y_real, ψ_real, x1_real, y1_real, x2_real, y2_real]) 
        u1_0 = u1_val
        u2_0 = u2_val

        #次のMPCに用いる微係数
        R_0_dash = minimum_error[1][2][1]
        X_vv_dash = minimum_error[1][2][2]
        X_vr_dash = minimum_error[1][2][3]
        X_rr_dash = minimum_error[1][2][4]
        X_vvvv_dash = minimum_error[1][2][5]
        Y_v_dash = minimum_error[1][2][6]
        Y_r_dash = minimum_error[1][2][7]
        Y_vvv_dash = minimum_error[1][2][8]
        Y_vvr_dash = minimum_error[1][2][9]
        Y_vrr_dash = minimum_error[1][2][10]
        Y_rrr_dash = minimum_error[1][2][11]
        N_v_dash = minimum_error[1][2][12]
        N_r_dash = minimum_error[1][2][13]
        N_vvv_dash = minimum_error[1][2][14]
        N_vvr_dash = minimum_error[1][2][15]
        N_vrr_dash = minimum_error[1][2][16]
        N_rrr_dash = minimum_error[1][2][17]
    end

    # 結果書き出し
    CSV.write("./results/MPC_result.csv", df_x)
end

run_mpc (generic function with 1 method)

In [27]:
N = 30 # 予測ホライゾン
duration = 100 # 全体の計算時間
dt = 0.5 # 刻み幅

run_mpc(duration, dt)

Progress: 100%|█████████████████████████████████████████| Time: 0:03:44


"./results/MPC_result.csv"